# Modelling to Generate Alternatives

In this example, we apply MGA ('modelling to generate alternatives') to a single node capacity expansion model in the style of [model.energy](https://model.energy).

The MGA algorithm, which can be called with `n.optimize.optimize_mga()`, tries to minimize or maximize investment or dispatch in (groups of) technologies within a set cost budget.

For instance, it can be used to minimize the amount of wind capacity while keeping costs within 5% of the cost-optimal solution in terms of system costs.

:::{note}
See also https://model.energy and [this paper](https://doi.org/10.1016/j.epsr.2020.106690) which uses PyPSA for MGA-type analysis.
:::

In [ ]:
import pandas as pd

import pypsa

## Solve example network to cost-optimality

Running MGA requires knowledge of what the total system costs are in the optimum. So first, we need to solve for the cost-optimal solution.

In [ ]:
n = pypsa.examples.model_energy()
n.optimize(solver_name="highs")
n.statistics.capex().sum() + n.statistics.opex().sum()

## Extract cost-optimal results

Optimal total system cost by technology:

In [ ]:
tsc = (
    pd.concat([n.statistics.capex(), n.statistics.opex()], axis=1).sum(axis=1).div(1e9)
)
optimal_cost = tsc.sum()
tsc

The optimised capacities in GW (GWh for `Store` component):

In [ ]:
n.statistics.optimal_capacity().div(1e3)

Energy balances on electricity side (in TWh):

In [ ]:
n.statistics.energy_balance(bus_carrier="electricity").sort_values().div(1e6)

Energy balances plot as time series (in MW):

In [ ]:
n.statistics.energy_balance.plot.area(linewidth=0, bus_carrier="electricity")

## Find lowest wind capacity within 5% cost slack

The function `n.optimize.optimize_mga` takes three main arguments:
1. The `slack` for the allowed relative cost deviation from the cost-optimum (0.05 corresponds to 5%).
2. A dictionary of weights for defining the new objective function. The first level defines the component (e.g. "Generator"), the second level defines the optimisation variable (e.g. `p_nom` for investment), and the third level defines the component name (e.g. from `n.generators.index`).
3. The `sense`, noting whether to minimizes ("min") or maximize ("max") the new objective.

In [ ]:
weights = {"Generator": {"p_nom": {"wind": 1}}}
n.optimize.optimize_mga(slack=0.05, weights=weights, sense="min", solver_name="highs")

The breakdown of total system costs shifts from wind towards more solar.

In [ ]:
tsc = (
    pd.concat([n.statistics.capex(), n.statistics.opex()], axis=1).sum(axis=1).div(1e9)
)
tsc

Up to numeric differences, it is 5% more expensive overall:

In [ ]:
optimal_cost * 1.05

In [ ]:
tsc.sum()

Overall, the wind capacity is cut by roughly a third:

In [ ]:
n.statistics.optimal_capacity().div(1e3)

This is also evident in the energy balance:

In [ ]:
n.statistics.energy_balance(bus_carrier="electricity").sort_values().div(1e6)

And also recognizable in the energy balance time series plots:

In [ ]:
n.statistics.energy_balance.plot.area(linewidth=0, bus_carrier="electricity")